# Parameter definition
Choose all the settings for the database creation.

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
db_name= 'test_video_dataset' #'colab_main_dataset'
input_dir = '/home/andrea/Desktop/image_datasets'
output_dir = '/home/andrea/Desktop/sql_database'
database_names=['lfw'] # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
detector_names=['mtcnn', 'mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender', 'age']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser-fiq'] # 'confusion_score',
save_in_drive = False # To work in google colab

## SERFIQ model
Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

2022-11-11 13:12:40.666615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[13:12:41] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[13:12:41] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[13:12:42] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[13:12:42] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upg

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/andrea/Desktop/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/andrea/Desktop/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f6734a750d0>,
 'databases': [<sql_face.databases.LFW at 0x7f6734a2ac50>],
 'detector_names': ['mtcnn', 'mtcnn_serfiq'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser-fiq']}

In [6]:
database.create_tables(serfiq=serfiq)

Creating image record from LFW: 0it [00:00, ?it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 0s 165ms/step


TRIM Creating Cropped Images for detector mtcnn:  20%|██        | 1/5 [00:02<00:10,  2.63s/it]

1/1 [==============================] - 0s 97ms/step


TRIM Creating Cropped Images for detector mtcnn:  40%|████      | 2/5 [00:04<00:06,  2.20s/it]

...

TRIM Creating Cropped Images for detector mtcnn:  60%|██████    | 3/5 [00:06<00:04,  2.09s/it]

1/1 [==============================] - 0s 75ms/step


TRIM Creating Cropped Images for detector mtcnn:  80%|████████  | 4/5 [00:08<00:02,  2.05s/it]

1/1 [==============================] - 0s 91ms/step


TRIM Creating Cropped Images for detector mtcnn: 100%|██████████| 5/5 [00:10<00:00,  2.07s/it]
TRIM Creating Cropped Images for detector mtcnn_serfiq:   0%|          | 0/5 [00:00<?, ?it/s][13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[13:12:54] 

In [7]:
database.update_tables(attributes_to_update, serfiq)

Update cropped images serfiq:   0%|          | 0/30 [00:18<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'apply_mtcnn'